In [1]:
import geemap
import ee

In [2]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
roi = Map.draw_last_feature.geometry()

In [ ]:
images = ee.ImageCollection("MODIS/006/MOD15A2H").filterDate('2015', '2021').select('Lai_500m')

In [ ]:
colorizedVis = {
  'min': 0,
  'max': 100,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
};

In [ ]:
img_n = images.size().getInfo()
img_n

In [ ]:
img_min = images.toBands().toArray().arraySort().arrayGet([img_n*10//100])
img_max = images.toBands().toArray().arraySort().arrayGet([img_n*90//100])

In [ ]:
img = ee.Image([img_max.clip(roi), img_min.clip(roi)])
img = ee.Image([img, img, img])

In [ ]:
Map.addLayer(img_min.clip(roi), colorizedVis, 'a')

In [ ]:
clip_dow_merge(roi, img_max, outfile=r'H:/Monarch/Data/Lai',
               scale=500, crs='EPSG:3857')

In [ ]:
clip_dow_merge(roi, img_min.clip(roi).unmask(), outfile=r'H:/Monarch/Data/Lai_mix',
               scale=500, crs='EPSG:3857', sep=0.05)

In [ ]:
clip_dow_merge(roi, img_max.clip(roi), outfile=r'H:/Monarch/Data/Lai_max',
               scale=500, crs='EPSG:3857', sep=0.05)

In [ ]:
def clip_dow_merge(geo: ee.Geometry, image: ee.Image, outfile: str, scale: int,
                   crs='epsg:4326', sep=0.25):
    """

    Args:
        geo: ee.Geometry, 需要下载的区域矢量几何
        image: ee.Image, 单波段影像
        outfile: str, 输出文件路径和名称，不需要文件后缀，下载的影响默认后缀为tif
        scale: int, 下载时的像元大小
        crs: str, 下载影像的投影，默认为 'epsg:4326' wgs1984投影
        sep: float, 单波段10m分辨率像元的影像裁剪大小(单位：经纬度)，默认为0.25
    Returns: None

    """
    import os
    import numpy as np
    import rasterio
    from glob import glob
    from rasterio.merge import merge
    import shutil
    bounds = geo.bounds()
    poy = np.array(bounds.coordinates().getInfo()[0])
    min_x = poy[:, 0].min()
    max_x = poy[:, 0].max()
    min_y = poy[:, 1].min()
    max_y = poy[:, 1].max()
    step = scale / 10 * sep
    end_x = int((max_x - min_x) / step) + 1
    end_y = int((max_y - min_y) / step) + 1
    polys = []
    for i in range(end_y):
        y1 = min_y + step * i
        y2 = min_y + step * (i + 1)
        if y2 > max_y:
            y2 = max_y
        for j in range(end_x):
            x1 = min_x + step * j
            x2 = min_x + step * (j + 1)
            if x2 > max_x:
                x2 = max_x
            poly = ee.Geometry(ee.Geometry.Rectangle([x1, y1, x2, y2]), None, False)
            polys.append(poly)
    if(len(polys) > 1):
        print(f"分割成{len(polys)}份, 开始下载:")
        path = outfile+'_mk'
        if not os.path.exists(path):
            os.makedirs(path)
        for j, i in enumerate(polys):
            geemap.ee_export_image(image, path+f'/temp_{j}.tif', scale=scale, crs=crs, region=i)
        files = glob(path+"/*.tif")
        src_files_to_mosaic = []
        for tif_f in files:
            src = rasterio.open(tif_f)
            src_files_to_mosaic.append(src)
        mosaic, out_trans = merge(src_files_to_mosaic)
        out_meta = src.meta.copy()
        out_meta.update({"driver": "GTiff",
                         "height": mosaic.shape[1],
                         "width": mosaic.shape[2],
                         "transform": out_trans,
                         })
        with rasterio.open(outfile+".tif", "w", **out_meta) as dest:
            dest.write(mosaic)
        for src in src_files_to_mosaic:
            src.close()
        shutil.rmtree(path)
    else:
        geemap.ee_export_image(image, outfile+'.tif', scale=scale, crs=crs, region=geo)
    print("download successful !!!")